In [19]:
import pandas as pd
import numpy as np 
import csv
import time

In [26]:
start = time.time()
path = '/home/manikya_varshney/Downloads/h01-20200903-204359().csv'
#data = pd.read_csv(path, index_col=None, header=0, engine='python' )
data = pd.read_csv(path, low_memory=False)
end = time.time()
print("Read parquet with pandas: ",(end-start),"sec")

Read parquet with pandas:  0.3315002918243408 sec


In [3]:
cols = [0, 1, 3, 4, 5,6, 7, 8, 14, 15, 16, 17, 18, 19, 22, 30, 31, 32, 34, 35, 36, 41, 42, 43, 45, 51, 52, 53, 54, 55, 56, 58, 63, 64, 65, 67, 73, 74, 75, 76, 77]
    
data = data[data.columns.values[cols]]

data_en = data[data['lang'] == 'en'].reset_index(drop=True)

In [4]:
data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'
interim = data_en[np.where((data_en['is_quote_tweet'] == False) & (data_en['RT_Flag']==False),True,False)].reset_index(drop=True)

interim['extended_tweet_full_text'] = np.where(~interim['truncated'], interim['text'], interim['extended_tweet_full_text'])

non_rep = interim[['created_at', 'id','user_id', 'coordinates', 'location', 'extended_tweet_full_text']]
non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

############### is_quote_tweet --> False --> RT ###############

interim = data_en[np.where((data_en['is_quote_tweet'] == False) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])

interim = interim[['created_at', 'RT_id','RT_user_id', 'RT_coordinates', 'RT_location','RT_full_text']]

interim = interim.drop_duplicates(['RT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

    ############### is_quote_tweet --> True --> Non RT ###############

interim = data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

interim['extended_tweet_full_text'] = np.where(~interim['truncated'],interim['text'],interim['extended_tweet_full_text'])
interim = interim[['created_at', 'id','user_id', 'coordinates', 'location', 'extended_tweet_full_text']]
interim.columns = non_rep.columns
interim = interim.drop_duplicates(['id']).reset_index(drop=True)

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

interim = data_en[np.where((data_en['is_quote_tweet'] == True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])

interim = interim[['created_at','QT_id','QT_user_id', 'QT_coordinates', 'QT_location', 'QT_full_text']]

interim = interim.drop_duplicates(['QT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)
    ############### is_quote_tweet --> True --> RT ###############

interim = data_en[np.where((data_en['is_quote_tweet'] == True) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])
    
interim = interim[['created_at', 'RT_id','RT_user_id', 'RT_coordinates', 'RT_location','RT_full_text']]

interim = interim.drop_duplicates(['RT_id']).reset_index(drop=True)

interim.columns = non_rep.columns
non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

interim = data_en[np.where((data_en['is_quote_tweet']== True ) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])
interim = interim[['created_at', 'QT_id','QT_user_id', 'QT_coordinates', 'QT_location', 'QT_full_text']]

interim = interim.drop_duplicates(['QT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

In [27]:
data['truncated'].value_counts()

False    17414
True      2523
Name: truncated, dtype: int64

In [28]:
data.shape

(19937, 79)

In [29]:
data['truncated'].isna().value_counts()

False    19937
Name: truncated, dtype: int64

In [24]:
nan_value = float("NaN")
data['truncated'].replace("", nan_value, inplace=True)

In [25]:
data.dropna(subset = ["truncated"], inplace=True)

In [18]:
data.to_csv('/home/manikya_varshney/Downloads/h01-20200903-204359.csv',index=False)